In [1]:
import numpy as np
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from stl import mesh
import trimesh
import pandas as pd
from IPython.display import clear_output
import vtkplotlib as vpl

In [2]:
# Setting plotting parameters
fs = 18
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rc('xtick',labelsize=fs)
plt.rc('ytick',labelsize=fs) 

In [ ]:
file = r"stl path"
msh = mesh.Mesh.from_file(file)
areas = msh.areas/1000/1000
unit_normals = msh.normals/np.linalg.norm(msh.normals,axis=1)[:,np.newaxis].copy()
centroids = msh.centroids.copy()

In [4]:

# Plot the mesh
fig = vpl.figure()
vpl.mesh_plot(msh, tri_scalars = areas, fig=fig)  # Using default styling
# vpl.arrow(msh.centroids[::50,:], msh.centroids[::50,:]+unit_normals[::50,:], color='red', fig=fig)
vpl.arrow(np.array([-15,0,0]), np.array([-15,0,0]) + 10*np.array([0,0,1]), color='green', fig=fig)
vpl.arrow(np.array([-15,0,0]), np.array([-15,0,0]) + 10*np.array([1,0,0]), color='blue', fig=fig)
vpl.arrow(np.array([-15,0,0]), np.array([-15,0,0]) + 10*np.array([0,1,0]), color='green', fig=fig)

# Show the figure
vpl.show()

## Newtonian Theory for Hypersonic Flow

Anderson 3.2:

$$\frac{F}{A} = \rho_{\infty}V_{\infty}^2sin^2(\theta)$$

Anderson 3.3

$$ \frac{p-p_{\infty}}{\frac{1}{2}\rho_{\infty}V_{\infty}^2} = 2sin^2(\theta) $$

$$ C_p = 2sin^2(\theta) $$

For 3D body (Anderson 3.5):

$$sin(\theta) = \frac{\vec V_{\infty} }{|\vec V_{\infty}|}\cdot \hat{n} $$

Where $\hat{n}$ is the unit vector normal to the surface of the body and $\vec V_{\infty}$ is the freestream velocity vector.

In [ ]:
c = 1 # Reference length, m

M_inf = 7.2 # Freestream Mach number
P0 = 250*6894.76 # Stagnamtion pressure, Pa
p_p0 =  0.00020188 # Mach 7 pressure ration
p_inf = p_p0*P0 # Static pressure
gamma = 1.4 # ratio of specific heats
q_inf = gamma/2*p_inf*M_inf**2 # Freestream dynamic pressure
D_ref = 1*25.4/1000 # TA diameter, meters
A_ref = np.pi*(D_ref/2)**2 # Reference area, m^2

Pitch = np.arange(-20,20,0.1)
CD = []
CL = []
for p in Pitch:
    V_hat_Original = np.array([1,0,0])
    C_p = np.zeros_like(unit_normals[:,0]) # Empty array for C_p


    pitch = p*np.pi/180 # Rotation angle around x-axis, rad
    yaw = 0*np.pi/180 # Rotation angle around y-axis, rad 
    roll = 0*np.pi/180 # Rotation angle around z-axis, rad 

    # Define rotation matrices
    R_x = np.array([[1,0,0], [0, np.cos(roll), -np.sin(roll)], [0, np.sin(roll), np.cos(roll)]])
    R_y = np.array([[np.cos(yaw), 0, np.sin(yaw)], [0, 1, 0], [-np.sin(yaw), 0, np.cos(yaw)]])
    R_z = np.array([[np.cos(pitch), -np.sin(pitch),0], [np.sin(pitch), np.cos(pitch),0],[0, 0, 1]])
    Rot_matrix = R_x@R_y@R_z # Combine rotaton matrices as needed

    pitch = -pitch # Rotation angle around x-axis, rad
    yaw = -yaw # Rotation angle around y-axis, rad 
    roll = -roll # Rotation angle around z-axis, rad 

    # Define rotation matrices
    R_x = np.array([[1,0,0], [0, np.cos(roll), -np.sin(roll)], [0, np.sin(roll), np.cos(roll)]])
    R_y = np.array([[np.cos(yaw), 0, np.sin(yaw)], [0, 1, 0], [-np.sin(yaw), 0, np.cos(yaw)]])
    R_z = np.array([[np.cos(pitch), -np.sin(pitch),0], [np.sin(pitch), np.cos(pitch),0],[0, 0, 1]])
    Rot_matrix2 = R_x@R_y@R_z # Combine rotaton matrices as needed

    V_hat = Rot_matrix@V_hat_Original
    #centroids = centroids - np.array([CGx,CGy,CGz])


    for i in range(len(unit_normals)):
            n_hat = unit_normals[i,:]
            C_p[i] = 2*np.max([np.dot( V_hat, n_hat ),0])**2#2*(np.dot( V_hat, n_hat))**2

    Fx = q_inf*np.trapz((C_p*(unit_normals[:,0])*areas.squeeze()))
    Fy = q_inf*np.trapz((C_p*(unit_normals[:,1])*areas.squeeze()))
    Fz = q_inf*np.trapz((C_p*(unit_normals[:,2])*areas.squeeze()))



    Cd = Fx/q_inf/A_ref
    Cl = Fy/q_inf/A_ref
    Cx = Fz/q_inf/A_ref
    
    CD.append(Cd)
    CL.append(Cl)
    
CD = np.array(CD)
CL = np.array(CL)

print('Fx = ' + str(round(Fx,3)))
print('Fy = ' + str(round(Fy,3)))
print('Fz = ' + str(round(Fz,3)))
print(' ')
print('Cd = ' + str(round(Cd,3)))
print('Cl = ' + str(round(Cl,3)))
print('Cx = ' + str(round(Cx,3)))

In [ ]:
plt.plot(Pitch,CD)
plt.plot(Pitch,CL)

plt.show()

In [21]:
df = pd.DataFrame(Pitch, columns = ['p'])
df['CD'] = CD
df['CL'] = CL

In [ ]:
df.head()

In [23]:
save_df = 1 # Chose to save the dataframe or not
df_file = 'Newtonian Aerocoefficients.xlsx'

# Saving run_matrix
if save_df == 1:
    df.to_excel(df_file)